```
  _____       _______ _____          _
 |  __ \     |__   __|  __ \        | |
 | |__) | __ ___| |  | |__) |__  ___| |_
 |  ___/ V__/ _ \ |  |  ___/ _ \/ __| __|
 | |   | | |  __/ |  | |  | (_) \__ \ |_
 |_|   |_|  \___|_|  |_|   \___/|___/\__|

```


# Installing PreTPost
Start by installing [nsjail](https://github.com/google/nsjail) in the `agent/nsjail` directory.
Make sure agents have these packages installed and accessible within the nsjail sandbox.

# Running the agent
The `sensitiveData.ini` file contains that sensitive data. To run the agent on the port 5000, run the following command:
```
cd agent
pkill -9 nsjail ; pkill -9 python ; clear; rm -rf instance* ; python3 agent.py --port 5000 --parallel 10
```

Use your browser to see the [analysis upload form](http://localhost:5000/form), [public policy](http://localhost:5000/publicpolicy) and current [total cost](http://localhost:5000/totalcost) and the result of previous analysis: [start here](http://localhost:5000/hello)

Private and public data and intermediate results are [listed](http://localhost:5000) for the debugging purpose.

# Running the curator
Run several agents on different machines or several instances on the same machine but using different ports and list their URI in the `agentlist.py` and run the following command.
```
python3 curatorRansomisedResponse.py
```

# Running an standalone analysis  
To modify and debug an analysis you can run it outside the sandbox:

```
export PYTHONPATH=$PYTHONPATH:FULLPATH/libdp/
```
Running the pre-processing script
```
cd curator/analysis/randomized-response/
python3 preproc.py  ../../../agent/sensitiveData.ini ./out.txt
```
Running  the post-processing script
```
cd curator/analysis/randomized-response/
python3 postproc.py  ./out.txt  outRandomised.txt 0.1
```

## Contribution
Contributions are welcome and appreciated. Please conform to [PEP8 coding style](http://www.python.org/dev/peps/pep-0008/).



# libdp Library
To generate a staircase matrix with the privacy cost of 2:

In [1]:
import libdp.libdp as dp
epsilon = 2
dpMatrix= dp.buildStairCaseMatrix(epsilon, ["-1", "1"])
print(dpMatrix)

attribute,-1,1
-1,7.38905609893065,1
1,1,7.38905609893065



After saving the content in the matrix.txt file, you can load it using loadTransitionMatrix() function.

In [1]:
with open("matrix.txt", "w") as tfile:
    print(dpMatrix, file=tfile)
m, f = dp.loadTransitionMatrix('matrix.txt')
print(f)

NameError: name 'dpMatrix' is not defined

In [3]:
print(m)

{('1', '-1'): 0.11920292202211755, ('-1', '1'): 0.11920292202211755, ('1', '1'): 0.8807970779778824, ('-1', '-1'): 0.8807970779778824}


In [4]:
print (dp.measureEpsilon(m, f))

2.0


# curatorlibdp Library
To send queries and get responses from agents

In [5]:
import sys,os
sys.path.append("curator/curatorlibdp")

import curator.curatorlibdp.curatorlibdp as cdp
analysis = cdp.ReqResList()
print (analysis)

[
reqId: uri:http://localhost:5000/ budget:9120.0 results:{}, 
reqId: uri:http://127.0.0.1:5000/ budget:9120.0 results:{}]


In [6]:
import sys
sys.path.append("curator/curatorlibdp")

import curator.curatorlibdp.curatorlibdp as cdp
analysis = cdp.ReqResList()
analysis.runIdenticalQuery(
        pre='curator/analysis/randomized-response/preproc.py',
        preParam=str(epsilon),
        timeout=2,
        cpu=2,
        typeDataList=[('analysesMatrix', dpMatrix)],
        post='curator/analysis/randomized-response/postproc.py',
        postParam=str(epsilon),
    )
print(analysis.ReqReslist)

[
reqId:1511188853487 uri:http://localhost:5000/ budget:9120.0 results:{0=1.3130352854993312,}, 
reqId:1511188853503 uri:http://127.0.0.1:5000/ budget:9120.0 results:{0=-1.3130352854993312,}]
